In [1]:
cd ../..

/home/wtaisner/PycharmProjects/reasonable-embeddings


In [2]:
from pathlib import Path
base_dir = Path('local/out/elpp/')

base_dir.mkdir(parents=True, exist_ok=True)

In [3]:
import lzma
import dill

with lzma.open(base_dir / 'reasoners.dill.xz', 'rb') as f:
    reasoners = dill.load(f)

In [12]:
from src.reasoner import ReasonerHead
from src.reasoner import EmbeddingLayer

with lzma.open(base_dir / 'exp1.dill.xz', 'rb') as f:
    artifacts: dict = dill.load(f)

emb_size = 32
hidden_size = 16

for key, components in artifacts.items():
    neural_reasoner = ReasonerHead(emb_size=emb_size, hidden_size=hidden_size)
    neural_reasoner.load_state_dict(components['reasoner'])
    components['reasoner'] = neural_reasoner
    encoders = [EmbeddingLayer(emb_size=emb_size, n_concepts=reasoner.n_concepts, n_roles=reasoner.n_roles) for reasoner in
                reasoners]
    for sd, e in zip(components['encoders'], encoders):
        e.load_state_dict(sd)
    components['encoders'] = encoders

In [13]:
with lzma.open(base_dir / 'test_reasoners.dill.xz', 'rb') as f:
    test_reasoners = dill.load(f)

In [14]:
best_neural_reasoner = artifacts[max(artifacts.keys())]['reasoner']

In [15]:
import numpy as np
from src.reasoner import ReasonerHead, EmbeddingLayer, train
from src.utils import timestr, paramcount
import torch as T
from src.elpp.gen import split_dataset

seed = 2022
ts = timestr()

emb_size = 32
hidden_size = 16
epoch_count = 15
test_epoch_count = 10
batch_size = 128

test_artifacts = {}

for complexity_threshold in range(2, 21):

    print("Complexity threshold", complexity_threshold)

    training, validation, test = split_dataset(test_reasoners, np.random.default_rng(seed=0xbeef), complexity_threshold=complexity_threshold)

    T.manual_seed(seed)
    reasoner = best_neural_reasoner
    encoders = [EmbeddingLayer(emb_size=emb_size, n_concepts=reasoner.n_concepts, n_roles=reasoner.n_roles) for reasoner in
                test_reasoners]

    print(f'created {len(encoders)} encoders with {paramcount(encoders[0])} parameters each')

    train_logger = train(training, validation, reasoner, encoders, epoch_count=epoch_count, batch_size=batch_size, freeze_reasoner=True)

    test_artifacts[complexity_threshold] = {
        'encoders': encoders,
        'training': training,
        'validation': validation,
        'test': test
    }


Complexity threshold 2
Training 10202 #pos 5101
Validation 2528 #pos 1264
Test 22480 #pos 22480
created 20 encoders with 7424 parameters each
train epoch 00/15 | batch 81/80 | loss 0.8342 | val loss 0.8140 | acc 0.7516 | f1 0.7422 | prec 0.7713 | recall 0.7152 | roc auc 0.8145 | pr auc 0.8621 | elapsed 0.90s
train epoch 01/15 | batch 81/80 | loss 0.8168 | val loss 0.7775 | acc 0.7575 | f1 0.7478 | prec 0.7789 | recall 0.7191 | roc auc 0.8217 | pr auc 0.8666 | elapsed 2.04s
train epoch 02/15 | batch 81/80 | loss 0.7428 | val loss 0.7409 | acc 0.7638 | f1 0.7538 | prec 0.7873 | recall 0.7231 | roc auc 0.8292 | pr auc 0.8714 | elapsed 2.23s
train epoch 03/15 | batch 81/80 | loss 0.6732 | val loss 0.7090 | acc 0.7702 | f1 0.7596 | prec 0.7962 | recall 0.7263 | roc auc 0.8361 | pr auc 0.8761 | elapsed 2.12s
train epoch 04/15 | batch 81/80 | loss 0.6112 | val loss 0.6810 | acc 0.7765 | f1 0.7670 | prec 0.8010 | recall 0.7358 | roc auc 0.8424 | pr auc 0.8804 | elapsed 2.52s
train epoch 05/15 

In [16]:
tmp = {key: {'encoders': [e.state_dict() for e in value['encoders']], 'training': value['training'], 'validation': value['validation'], 'test': value['test']} for key, value in test_artifacts.items()}

with lzma.open(base_dir / 'exp2.dill.xz', 'wb') as f:
    dill.dump(tmp, f)

In [17]:
from tqdm import tqdm
from src.reasoner import eval_batch

import pandas as pd

rows = []

for complexity_threshold, components in tqdm(artifacts.items()):
    with T.no_grad():
        idx_te, X_te, y_te = components['test']
        _, _, Y_te_good = eval_batch(best_neural_reasoner, components['encoders'], X_te, y_te, idx_te)
    for i in range(len(idx_te)):
        idx = idx_te[i]
        axiom = X_te[i]
        expected = y_te[i]
        predicted = Y_te_good[i]
        complexity = len(reasoners[idx].decode_shortest_proof(axiom[1], axiom[2]))
        rows.append([complexity_threshold, idx, complexity, axiom, expected, int(predicted >= .5), predicted])

100%|██████████| 19/19 [00:24<00:00,  1.27s/it]


In [18]:
df = pd.DataFrame(rows, columns=["Complexity threshold", "KB", "Complexity", "Axiom", "Expected", "Predicted", "Raw predicted"])
df.to_feather(base_dir / 'exp2.feather')
df


,Complexity threshold,KB,Complexity,Axiom,Expected,Predicted,Raw predicted
0,2,0,4,"(0, 1, 9)",1,1,0.762662
1,2,0,6,"(0, 1, 50)",1,1,0.992152
2,2,0,5,"(0, 1, 55)",1,0,0.083703
3,2,0,7,"(0, 1, 63)",1,1,0.974874
4,2,0,5,"(0, 1, 65)",1,0,0.440781
...,...,...,...,...,...,...,...
445205,20,39,24,"(0, 89, 44)",1,0,0.003385
445206,20,39,22,"(0, -2, 49)",1,0,0.003953
445207,20,39,24,"(0, -2, 56)",1,0,0.055731
445208,20,39,25,"(0, -2, 86)",1,0,0.024811
